In [0]:
query = """
SELECT
    ROW_NUMBER() OVER (ORDER BY ci.cid) AS customer_key,
    ci.cid AS customer_id,
    ci.first_name,
    ci.last_name,
    la.country,
    ci.marital_status,
    CASE
        WHEN ci.gender <> 'n/a' THEN ci.gender
        ELSE COALESCE(ca.gender, 'n/a')
    END AS gender,
    ca.birth_date AS birthdate,
    ci.created_date AS create_date
FROM (
    SELECT
        CAST(customer_id AS STRING) AS cid,
        *
    FROM silver.crm_customers
) ci
LEFT JOIN (
    SELECT
        CAST(customer_number AS STRING) AS customer_number,
        gender,
        birth_date
    FROM silver.erp_customers
) ca
    ON ci.cid = ca.customer_number
LEFT JOIN (
    SELECT
        CAST(customer_number AS STRING) AS customer_number,
        country
    FROM silver.erp_customer_location
) la
    ON ci.cid = la.customer_number
"""
df = spark.sql(query)

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("workspace.gold.dim_customers")